# BoW Mineria de Datos


*   Estudiante: Inca Cruz Carlos Eduardo

# Instalar pyspark

In [ ]:
#get package
!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar -xvzf spark-3.2.0-bin-hadoop2.7.tgz
!pip install findspark
#env vars
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
#start spark
import findspark
findspark.init()
#import spark session
from pyspark.sql import SparkSession
#build app
spark = SparkSession.builder.appName("PySpark").getOrCreate()
print(spark.sparkContext.appName)
#create sparkcontext
sc=spark.sparkContext
#clear output
from google.colab import output
output.clear()

# Baf Of Words

**1.Bag Of Words**

In [12]:
import re
def RemoverCaracteres(texto):
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()

In [17]:
import os.path
import operator

import re
# Remover Caracteres
def RemoverCaracteres(texto):
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
# Leer archivo de texto
archivo = os.path.join('.', 'pg100.txt') 

#introducir el texto
shakesRDD = (sc.textFile(archivo, 8).map(RemoverCaracteres))
def Tokenizar(lista):
    return [i for i in lista.split(" ")]

shakesRDD = shakesRDD.flatMap(Tokenizar)
# Eliminar palabras vacias
shakesRDD = shakesRDD.filter(lambda x : len(x) !=0)
# Eliminar palabras con longitudes menores a dos
shakesRDD = shakesRDD.filter(lambda x : len(x) > 2)
# Asignar el 1 a cada palabra para contar la frecuencia
shakesRDD = shakesRDD.map(lambda x : (x,1))
# Eliminar palabras repetidas
shakesRDD = shakesRDD.reduceByKey(lambda a,b: a+b)
# Obtener las claves
shakesRDD = shakesRDD.map(lambda x: x[0])
# Tomar solo 50 datos
print(shakesRDD.take(50))

['other', 'whatsoever', 'gutenberg', 'are', 'check', 'where', 'using', '2017', 'february', 'set', 'encoding', 'start', 'ends', 'cleopatra', 'like', 'errors', 'sixth', 'john', 'julius', 'lost', 'macbeth', 'nights', 'ado', 'othello', 'twelfth', 'two', 'passionate', 'thereby', 'but', 'tender', 'selfsubstantial', 'fuel', 'self', 'now', 'when', 'besiege', 'praise', 'deservd', 'sum', 'succession', 'repair', 'unbless', 'husbandry', 'why', 'free', 'beauteous', 'bounteous', 'profitless', 'yet', 'work']


**2.Obtener frecuencia**

In [19]:
import os.path
import operator
# Leer archivo de texto
archivo = os.path.join('.', 'pg100.txt') 

#introducir el texto
shakesRDD = (sc.textFile(archivo, 8).map(RemoverCaracteres))

shakesRDD = shakesRDD.flatMap(Tokenizar)
# Eliminar palabras vacias
shakesRDD = shakesRDD.filter(lambda x : len(x) !=0)
# Eliminar palabras con longitudes menores a dos
shakesRDD = shakesRDD.filter(lambda x : len(x) > 2)
# Asignar el 1 a cada palabra para contar la frecuencia
shakesRDD = shakesRDD.map(lambda x : (x,1))
# Obtener la frecuencia
shakesRDD = shakesRDD.reduceByKey(lambda a,b: a+b)
#Mostrar las palabras del bow
shakesRDD = shakesRDD.map(lambda x: (x[0],x[1]))
# Tomar solo 50 datos
print(shakesRDD.take(50))

[('other', 723), ('whatsoever', 17), ('gutenberg', 36), ('are', 3730), ('check', 32), ('where', 1372), ('using', 14), ('2017', 3), ('february', 2), ('set', 509), ('encoding', 1), ('start', 39), ('ends', 54), ('cleopatra', 264), ('like', 1928), ('errors', 15), ('sixth', 37), ('john', 534), ('julius', 19), ('lost', 287), ('macbeth', 285), ('nights', 71), ('ado', 20), ('othello', 326), ('twelfth', 3), ('two', 758), ('passionate', 11), ('thereby', 27), ('but', 6776), ('tender', 156), ('selfsubstantial', 1), ('fuel', 5), ('self', 144), ('now', 3011), ('when', 2225), ('besiege', 11), ('praise', 191), ('deservd', 39), ('sum', 58), ('succession', 17), ('repair', 48), ('unbless', 1), ('husbandry', 15), ('why', 1552), ('free', 204), ('beauteous', 44), ('bounteous', 19), ('profitless', 3), ('yet', 1754), ('work', 236)]


**3.Obtener N-Gramas**

In [20]:
def obtenNGramas(sentence, n):
    l1=[x for x in sentence.split(" ")]
    return [l1[i:i+n] for i in range(len(l1)-(n-1))]


In [21]:
dataRDD=sc.parallelize(['this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it'],8)
dataRDD2 = dataRDD.map(lambda x: (x,str(obtenNGramas(x,3))))
print(dataRDD2.collect())


[('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]")]


**4.Obtener TF IDF**

In [22]:

#Modulo remover Signos de puntuacion
import re
import math
from pyspark.sql.functions import *
import os.path


#almacenamos nuestros datos en un arreglo con un indice que nos indique  el numero de linea en el cual se encuentra
data=[(1,"messi messi messi ronaldo ronaldo balon"),(2,"messi ronaldo futbol futbol futbol")]
# Los elementos de la lista se copian para formar un conjunto de datos distribuido que se puede operar en paralelo
lineas=sc.parallelize(data)




# Contar la frecuencia de aparicion de cada palabra 
# en la oracion en una determinada linea
frecuenciaRDD=lineas.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
palabraoracion=frecuenciaRDD.reduceByKey(lambda x,y:x+y)


# Diccionario de palabras totales para hallar el tf
# Tiene como clave la palabra(token) y como valor
# el numero de veces que aparece en todos los documentos
diccionariotokens=lineas.flatMap(lambda x: [(i,1) for i in x[1].split()])
mpdic=diccionariotokens.reduceByKey(lambda x,y:x+y)
mpdict=dict(mpdic.collect())
# Mostrando frecuencia de aparicion de palabras en todo el documento


# Una vez obtenido el numero de veces que aparece una 
# palabra en todo el documento y el numero de palabras
# que tiene el documento
# Hallamos el tf con la formula
#tf=reduce.map(lambda x: (x[0][1],(x[0][0],1+math.log10(x[1]/mpdict[x[0][1]]))))
tf=palabraoracion.map(lambda x: (x[0][1],(x[0][0],1+math.log10(x[1]/mpdict[x[0][1]]))))
tfshow=tf.map(lambda x: (x[0],x[1][1]))
#Aquí, la clave será el token y su valor será el identificador TF del 
#documento para ese token junto con un contador de 1
frecp=palabraoracion.map(lambda x: (x[0][1],(x[0][0],x[1],1)))




#Obtenemos en cuantos documentos aparece una determinada palabra
map4=frecp.map(lambda x:(x[0],x[1][2]))
palabrapordocumento=map4.reduceByKey(lambda x,y:x+y)


#Una vez obtenidos en cuantos documentos aparece una palabra en x[1]
# y el numero total de documentos en el corpus en len(data)
idf=palabrapordocumento.map(lambda x: (x[0],math.log10 (1+len(data)/x[1]) ))


tfshow.toDF(["Token","TF"]).show()
idf.toDF(["Token","IDF"]).show()
#Unimos los valores de tf con itf con la funcion join
rdd=tf.join(idf)

# multiplicamos tf*itf
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()

# Formatemos para mostrar 
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

rdd.toDF(["Nro de Linea","Palabra","TF","IDF","TF*IDF"]).show()

+-------+------------------+
|  Token|                TF|
+-------+------------------+
|ronaldo|0.8239087409443188|
|  balon|               1.0|
|  messi|0.3979400086720376|
|  messi|0.8750612633917001|
|ronaldo|0.5228787452803375|
| futbol|               1.0|
+-------+------------------+

+-------+-------------------+
|  Token|                IDF|
+-------+-------------------+
|  messi| 0.3010299956639812|
|ronaldo| 0.3010299956639812|
|  balon|0.47712125471966244|
| futbol|0.47712125471966244|
+-------+-------------------+

+------------+-------+------------------+-------------------+-------------------+
|Nro de Linea|Palabra|                TF|                IDF|             TF*IDF|
+------------+-------+------------------+-------------------+-------------------+
|           1|ronaldo|0.8239087409443188| 0.3010299956639812|0.24802124471398448|
|           1|  messi|0.8750612633917001| 0.3010299956639812| 0.2634196883245214|
|           1|  balon|               1.0|0.477121254719662